In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, roc_curve,roc_auc_score,auc, f1_score
from sklearn.feature_selection import SelectFromModel


In [7]:
#formatage des données de training


df_train = pd.read_csv('Train_transformed.csv',  sep=';')

df_submission_test = pd.read_csv('Test_submission_transformed.csv', sep=';')

print("shape des données du Train.csv : {}".format(df_train.shape))
print("shape des données de Test.csv : {}".format(df_submission_test.shape))



X = df_train.drop('return90',axis=1)
y = df_train['return90']


shape des données du Train.csv : (78074, 689)
shape des données de Test.csv : (38992, 689)


In [13]:
print(X.shape)

rf_reduc = RandomForestClassifier()

rf_reduc.fit(X, y)

model_reduc = SelectFromModel(rf_reduc, prefit=True)

X_new = model_reduc.transform(X)

print(X_new.shape)


(78074, 688)


/Users/jerem/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


(78074, 29)


In [14]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

#df_sample_1000 = pre_select_sample.sample(n=50000, random_state=20) 


X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state= 20)


# nombre d'arbre
n_estimators = [100,200,300]

# nombre de features à chaque split
max_features = ['auto', 'sqrt']
# nombre max d'etage pour chaque arbre
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# nombre mini pour split un noeud
min_samples_split = [2, 5, 10]
# nombre mini pour chaque feuille
min_samples_leaf = [1, 2, 4]
# methode de selection des sous ensemble du dataset
bootstrap = [True, False]

#grille pour les test des parametres optimaux

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}




rf = RandomForestClassifier()

# 3 folds, 10 iterations = 30 test = env 8min sur le datasets sans reduction de dimensionnalités
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit le model
rf_random.fit(X_train, y_train)


print(rf_random.best_params_)






#print("début training randomforest")

#clf_rf = RandomForestClassifier(n_estimators = 5,min_samples_split=2, min_samples_leaf = 2, class_weight="balanced")
#clf_rf.fit(X_train,y_train)


#print("fin training")

#y_predict_sample = clf_rf.predict_proba(X_test)[:,1]




#fpr, tpr, _ = roc_curve(y_test,y_predict_sample)
#plt.plot(fpr, tpr, marker='.', label='LogistiIIc')


#print("AUC: {}".format(roc_auc_score(y_test, y_predict_sample)))


#reduction de dimensionnalité
#peut etre à utiliser pour un SVC

#print(X.shape)
#logreg_reduc = LogisticRegression(penalty="l1")
#logreg_reduc.fit(X, y)
#model_reduc = SelectFromModel(logreg_reduc, prefit=True)
#X_new = model_reduc.transform(X)
#print(X_new.shape)


#X_train, X_test, y_train, y_test = train_test_split(X_new,y,test_size=0.3, random_state= 20)
#best_p = tree_cv.best_params_
#rf = SVC(gamma='auto',probability=True)

#rf.fit(X_train,y_train)

#y_rf_p = rf.predict_proba(X_test)[:,1]

#print(y_rf_p)

#fpr, tpr, _ = roc_curve(y_test,y_rf_p)
#plt.plot(fpr, tpr, marker='.', label='LogistiIIc')


#print("AUC: {}".format(roc_auc_score(y_test, y_rf_p)))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.8min finished


{'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 110, 'bootstrap': False}


In [11]:
#reduction de dimensionnalité


print(X.shape)

rf_reduc = RandomForestClassifier(n_estimators = 300,min_samples_split=5, min_samples_leaf = 4,
                                max_features= 'sqrt', max_depth = 40, bootstrap = False)

rf_reduc.fit(X, y)

model_reduc = SelectFromModel(rf_reduc, prefit=True)

X_new = model_reduc.transform(X)

print(X_new.shape)



(78074, 688)
(78074, 42)


In [ ]:
#random forest avec les parametres tunés

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= 5)

print("début training randomforest")

clf_rf = RandomForestClassifier(n_estimators = 300,min_samples_split=5, min_samples_leaf = 4,
                                max_features= 'sqrt', max_depth = 110, bootstrap = False, class_weight= "balanced")


clf_rf.fit(X_train,y_train)


print("fin training")

y_predict = clf_rf.predict_proba(X_test)[:,1]




fpr, tpr, _ = roc_curve(y_test,y_predict)
plt.plot(fpr, tpr, marker='.', label='LogistiIIc')


print("AUC: {}".format(roc_auc_score(y_test, y_predict)))



début training randomforest


In [ ]:
#instance du model
#peut etre tune le meilleur solver, sachant que lgbs supporte seulement une norme L2
#norme L1 ou L2 ?

#faire une selection de variable avant de tuné le modele

logreg = LogisticRegression(penalty="l2", solver= 'lbfgs')

c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space}

logreg_cv = GridSearchCV(logreg, param_grid, cv=5)


logreg_cv.fit(X_train,y_train)

#Meilleur parametre C
bestparam = logreg_cv.best_params_


print("Parametre C tuné du model: {}".format(bestparam)) 
print("Meilleur score {}".format(logreg_cv.best_score_))


In [ ]:
#model de la submission 1 sur kaggle

clf_logreg = LogisticRegression(penalty="l1")
clf_logreg.fit(X,y)



fpr, tpr, _ = roc_curve(y_test,y_pred)
plt.plot(fpr, tpr, marker='.', label='Logistic')

print("AUC: {}".format(roc_auc_score(y_test, y_pred)))


#applique le modèle sur les données de test du Test.csv à envoyé à Kaggle
#choper la proba
#peut etre revoir le treshold mais comme on veut la proba dans ce cas la ca pose pas trop de pb pour le moment
df_X_sub_split_dropped = df_X_sub_split.drop('return90',axis=1)

y_pred = clf_logreg.predict_proba(df_X_sub_split_dropped)[:,1]
print(y_pred)


index = df_X_sub_split_dropped['booking_id'].astype(np.int64)
submission_df = pd.DataFrame({'return90': y_pred},index=index)


#submission_df.to_csv('submission1.csv',sep=',')

In [40]:
X_df_sub = df_submission_test.drop('return90', axis = 1)

X_df_sub_reduc = X_df_sub.loc[:, model_reduc.get_support()]

y_pred_sub = clf_rf.predict_proba(X_df_sub)[:,1]
print(y_pred_sub)


index = X_df_sub_reduc['booking_id'].astype(np.int64)
submission_df = pd.DataFrame({'return90': y_pred_sub},index=index)

submission_df.to_csv('submission2.csv',sep=',')

[0.52608365 0.53618308 0.39018932 ... 0.22911772 0.15339351 0.15448979]
